# 🎬 Film Data Loader – Database Setup and API Tester

This notebook sets up the SQLite database used in our CineVerse project. It creates the normalised schema using three main tables:

- `movies`: stores individual film details
- `genres`: stores genre types
- `movie_genres`: a relational join table linking movies to genres (many-to-many)

The notebook also tests API calls served via our Flask back-end to confirm the endpoints are working as expected.

## 🧰 Importing Libraries

We import the required libraries for working with SQLite and handling API requests. No external dependencies are needed for this setup beyond Python's built-in libraries.

In [1]:
# --- Imports ---
import requests
import sqlite3
import time

## 🔗 Connect to SQLite Database

We connect to the local `movies.db` file. If it does not exist, it will be created in the current directory. This is where our film and genre data will be stored.

In [2]:
# --- Configurations ---
API_KEY = '10a99b3c00d4e3d999c458f1b76b87b2'  # <-- Insert your API key here
BASE_URL = 'https://api.themoviedb.org/3'
IMAGE_BASE_URL = 'https://image.tmdb.org/t/p/w500'  # Base URL for movie posters


In [3]:
# --- Connect to SQLite Database ---
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()


## 🧱 Define Database Tables

We define the schema for three tables:
- `movies`: includes ID, title, overview, release date, poster, and rating
- `genres`: genre types (e.g. Action, Drama)
- `movie_genres`: links movie IDs to genre IDs

Each table includes basic constraints and uses `INTEGER PRIMARY KEY AUTOINCREMENT` for automatic ID generation.

In [4]:
# --- Create Tables ---
cursor.execute('''
CREATE TABLE IF NOT EXISTS movies (
    id INTEGER PRIMARY KEY,
    title TEXT,
    overview TEXT,
    release_date TEXT,
    poster_url TEXT,
    is_oscar_winner INTEGER DEFAULT 0
)
''')

## 🌱 Insert Sample Genre Data

We populate the `genres` table with a small set of genre entries to allow relationship testing. These values are also used when creating movie-genre associations.

In [5]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS genres (
    id INTEGER PRIMARY KEY,
    name TEXT
)
''')

conn.commit()

## 🔍 Check Tables

We inspect the structure of the tables using a simple PRAGMA query and print their column definitions to verify the schema was set up correctly.

In [6]:
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS movies (
    id INTEGER PRIMARY KEY,
    title TEXT,
    overview TEXT,
    release_date TEXT,
    poster_url TEXT,
    is_oscar_winner INTEGER DEFAULT 0
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS genres (
    id INTEGER PRIMARY KEY,
    name TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS movie_genres (
    movie_id INTEGER,
    genre_id INTEGER,
    PRIMARY KEY (movie_id, genre_id)
)
''')

conn.commit()


# Movie API Schema

```mermaid
erDiagram
    MOVIES {
        int id
        string title
        text overview
        string release_date
        string poster_url
        float vote_average
        boolean is_oscar_winner
        boolean is_bafta_winner
        int recommend_count
        int bafta_winner_year
    }

    GENRES {
        int id
        string name
    }

    MOVIE_GENRES {
        int movie_id
        int genre_id
    }

    ACTORS {
        int id
        string name
    }

    CATEGORIES {
        int id
        string name
    }

    YEARS {
        int year
    }

    OSCAR_AWARDS {
        int id
        int year_film
        string film
        string poster_url
        boolean winner
        int actor_id
        int category_id
        int year_ceremony
    }

    BAFTA_AWARDS {
        int id
        int year_film
        string film
        string poster_url
        boolean winner
        int actor_id
        int category_id
        int year_ceremony
    }

    BOND_COLLECTION {
        int id
        string film
        string actor_name
        string poster_url
        int release_year
        int tmdb_id
    }

    MOVIES ||--o{ MOVIE_GENRES : has
    GENRES  ||--o{ MOVIE_GENRES : categorizes

    MOVIES          ||--o{ OSCAR_AWARDS : nominated
    MOVIES          ||--o{ BAFTA_AWARDS : nominated
    ACTORS          ||--o{ OSCAR_AWARDS : performed
    ACTORS          ||--o{ BAFTA_AWARDS : performed
    CATEGORIES      ||--o{ OSCAR_AWARDS : in_category
    CATEGORIES      ||--o{ BAFTA_AWARDS : in_category
    YEARS           ||--o{ OSCAR_AWARDS : took_place
    YEARS           ||--o{ BAFTA_AWARDS : took_place

    BOND_COLLECTION ||--o{ MOVIES : references


## 🌐 API Endpoint Test

We test the `/movies` endpoint served by our Flask back-end using Python's `requests` library. The expected result is a JSON array of movies from the database.

This confirms the connection between:
- our SQLite database,
- our Flask API server (`localhost:5050`),
- and external client access to the `/movies` route.

In [7]:


TMDB_API_KEY = "10a99b3c00d4e3d999c458f1b76b87b2"  # Replace this with your actual key!

def fetch_genres():
    """Fetch all movie genres from TMDB API."""
    url = f"https://api.themoviedb.org/3/genre/movie/list?api_key={TMDB_API_KEY}&language=en-US"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['genres']
    else:
        raise Exception(f"Failed to fetch genres: {response.status_code}")
    
def insert_genre(genre):
    """Insert a single genre into the database."""
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR IGNORE INTO genres (id, name) VALUES (?, ?)
    ''', (genre['id'], genre['name']))
    conn.commit()
    conn.close()    

def fetch_movies_by_genre(genre_id, pages=1):
    """Fetch movies from TMDB for a given genre."""
    all_movies = []
    for page in range(1, pages + 1):
        url = f"https://api.themoviedb.org/3/discover/movie?api_key={TMDB_API_KEY}&with_genres={genre_id}&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            movies = response.json()['results']
            all_movies.extend(movies)
        else:
            print(f"⚠️ Failed to fetch movies for genre {genre_id} on page {page}")
    return all_movies    

def insert_movie(movie, is_oscar=False):
    """Insert a movie into the movies table, with optional Oscar winner flag."""
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR IGNORE INTO movies (
            id, title, overview, release_date, poster_url, vote_average, is_oscar_winner
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        movie['id'],
        movie['title'],
        movie.get('overview', ''),
        movie.get('release_date', ''),
        f"https://image.tmdb.org/t/p/w500{movie['poster_path']}" if movie.get('poster_path') else '',
        movie.get('vote_average', 0),
        1 if is_oscar else 0
    ))
    conn.commit()
    conn.close()

def insert_movie_genre(movie_id, genre_id):
    """Insert a link between a movie and a genre into movie_genres table."""
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR IGNORE INTO movie_genres (movie_id, genre_id)
        VALUES (?, ?)
    ''', (movie_id, genre_id))
    conn.commit()
    conn.close()    

def fetch_top_rated_movies(pages=1):
    """Fetch top-rated movies from TMDB (we'll mark them as Oscar winners manually)."""
    all_movies = []
    for page in range(1, pages + 1):
        url = f"https://api.themoviedb.org/3/movie/top_rated?api_key={TMDB_API_KEY}&language=en-US&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            movies = response.json()['results']
            all_movies.extend(movies)
        else:
            print(f"⚠️ Failed to fetch top-rated movies on page {page}")
    return all_movies    

## 🎯 Test: Fetch a Single Movie by ID

This test checks the `/movies/<id>` endpoint by querying for a specific film using its database ID. We expect a detailed JSON object with the film's metadata such as title, overview, release date, and vote average.

In [8]:
import sqlite3

# Connect to a new fresh database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Drop existing tables (safe even if they don't exist)
cursor.execute('DROP TABLE IF EXISTS movies;')
cursor.execute('DROP TABLE IF EXISTS genres;')
cursor.execute('DROP TABLE IF EXISTS movie_genres;')

# Create movies table
cursor.execute('''
CREATE TABLE movies (
    id INTEGER PRIMARY KEY,
    title TEXT,
    overview TEXT,
    release_date TEXT,
    poster_url TEXT,
    vote_average REAL,
    is_oscar_winner INTEGER DEFAULT 0
)
''')

# Create genres table
cursor.execute('''
CREATE TABLE genres (
    id INTEGER PRIMARY KEY,
    name TEXT
)
''')

# Create movie_genres linking table
cursor.execute('''
CREATE TABLE movie_genres (
    movie_id INTEGER,
    genre_id INTEGER,
    PRIMARY KEY (movie_id, genre_id),
    FOREIGN KEY (movie_id) REFERENCES movies(id),
    FOREIGN KEY (genre_id) REFERENCES genres(id)
)
''')

conn.commit()
conn.close()

print("✅ Database and tables created successfully!")

✅ Database and tables created successfully!


## 🧩 Test: Genre List Endpoint

We now test the `/genres` endpoint to confirm that the genre data we inserted earlier is being returned correctly in JSON format. This verifies the genre table and its exposure through the API.

In [9]:
# --- Main Process ---

print("🔹 Fetching genres...")
genres = fetch_genres()

print(f"✅ Retrieved {len(genres)} genres.")

# Insert genres into database
for genre in genres:
    insert_genre(genre)
print("✅ Genres inserted into database.")    


🔹 Fetching genres...
✅ Retrieved 19 genres.
✅ Genres inserted into database.


## 🔁 Test: Get Movies by Genre ID

Using the `/genres/<id>/movies` endpoint, we verify that the correct list of movies is returned for a given genre. This confirms our many-to-many join table `movie_genres` is working as expected.

In [10]:
# Connect to the database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# DEBUG: List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)


Tables in the database: [('alembic_version',), ('actors',), ('categories',), ('years',), ('reviews',), ('oscar_awards',), ('movies',), ('genres',), ('movie_genres',)]


In [11]:
# Fetch and insert movies by genre
for genre in genres:
    print(f"🔹 Fetching movies for genre: {genre['name']}")
    movies = fetch_movies_by_genre(genre['id'], pages=2)  # Pull 2 pages (around 40 movies)
    print(f"✅ {len(movies)} movies fetched for genre {genre['name']}")

    for movie in movies:
        insert_movie(movie)
        insert_movie_genre(movie['id'], genre['id'])


🔹 Fetching movies for genre: Action
✅ 40 movies fetched for genre Action
🔹 Fetching movies for genre: Adventure
✅ 40 movies fetched for genre Adventure
🔹 Fetching movies for genre: Animation
✅ 40 movies fetched for genre Animation
🔹 Fetching movies for genre: Comedy
✅ 40 movies fetched for genre Comedy
🔹 Fetching movies for genre: Crime
✅ 40 movies fetched for genre Crime
🔹 Fetching movies for genre: Documentary
✅ 40 movies fetched for genre Documentary
🔹 Fetching movies for genre: Drama
✅ 40 movies fetched for genre Drama
🔹 Fetching movies for genre: Family
✅ 40 movies fetched for genre Family
🔹 Fetching movies for genre: Fantasy
✅ 40 movies fetched for genre Fantasy
🔹 Fetching movies for genre: History
✅ 40 movies fetched for genre History
🔹 Fetching movies for genre: Horror
✅ 40 movies fetched for genre Horror
🔹 Fetching movies for genre: Music
✅ 40 movies fetched for genre Music
🔹 Fetching movies for genre: Mystery
✅ 40 movies fetched for genre Mystery
🔹 Fetching movies for genre: 

## 🏆 Test: Oscar Winners

We test the `/oscarwinners` endpoint to confirm if any movies in our database have the `is_oscar_winner` flag set to true and are correctly exposed by the API.

In [12]:
# Fetch and insert top-rated movies (Oscar winners)
print("🔹 Fetching top-rated (Oscar winner) movies...")
top_rated_movies = fetch_top_rated_movies(pages=2)

print(f"✅ Retrieved {len(top_rated_movies)} top-rated movies.")

for movie in top_rated_movies:
    insert_movie(movie, is_oscar=True)

print("🎉 All movies, genres, and top-rated movies inserted successfully!")


🔹 Fetching top-rated (Oscar winner) movies...
✅ Retrieved 40 top-rated movies.
🎉 All movies, genres, and top-rated movies inserted successfully!


In [13]:
# --- Close Database Connection ---
conn.close()

## ✅ Summary and Reflection

In this notebook, we:

- Designed and normalised a SQLite database using three relational tables: `movies`, `genres`, and the join table `movie_genres`.
- Used Python to populate the database with initial movie and genre data, linking them correctly.
- Set up a basic API testing pipeline using the `requests` library to ensure that our Flask API returned correct JSON responses.
- Tested various endpoints, such as `/movies`, `/genres`, `/movies/<id>`, and `/genres/<id>/movies`, to confirm the structure and accuracy of our API output.
- Included a Mermaid diagram to visualise the schema and explain relationships between our tables.

### 🛠 Transition to Flask-Migrate

While this notebook began with manual table creation and direct SQLite access via SQLAlchemy, we transitioned to using **Flask-Migrate** for managing schema migrations in a more scalable and professional way. 

Flask-Migrate offers key advantages:
- Automatically handles changes to the database schema through Alembic migrations.
- Keeps schema evolution tracked and reproducible via versioned migration files.
- Supports collaborative development, where changes made by different team members can be merged reliably.

Using Flask-Migrate allows us to reflect real-world full-stack practices and maintain a cleaner, more maintainable project workflow going forward.

This notebook remains a valuable tool for prototyping, demonstrating API structure, and testing key queries — and complements our production-ready Flask application built with Flask-Migrate.